In [1]:
import pandas as pd
from dynaconf import LazySettings
from dynaconf.utils.boxing import DynaBox
from typing import List
import os

In [2]:
config_file = "/home/tiziano/workspaces/fantasAi_football/config/conf.yaml"
config_mode = 'default'

In [3]:
def data_ingestion_basics(params: DynaBox, table: str, duplicate_key_err:str='raise') -> pd.DataFrame:
    import os 
    
    """Performs some basic harmonization steps on a input dataframe.

    Note that after the basic harmonization the columns will have the names
    stated in settings.COLS, and not those appearing in the input data.

    Args:
        table (pd.DataFrame): id of the table to lead, it must appear in
            config file
        duplicate_key_err ('raise' or 'drop'): how to handle duplicated key
        errors. If 'raise' an error will be return in case of duplicated keys,
        if 'drop' all duplicated rows will be dropped from data.

    Raises:
        ValueError: If table registry dataframe keys are missing
        KeyError: If table registry dataframe keys are duplicated

    Returns:
        pd.DataFrame: dataframe in input, harmonized
    """
    # Load parameters
    t_par = params[table]
    t_cols_dict = t_par["COLS"].to_dict()
    keys: List[str] = [t_par["COLS"][c] for c in t_par.KEY]
    cols: List[str] = list(t_cols_dict.values())
    dtype_map = {
        c_name: params["FEATURES"]["DTYPES"][c]
        for c, c_name in t_cols_dict.items()
    }
    name_conversion = {
        c_from: params["FEATURES"][c] for c, c_from in t_cols_dict.items()
    }
    
    path = os.path.join(
        params["PATHS"]["ROOT_FOLDER"], 
        params["PATHS"]["INPUT"]["FOLDER"],
         params["PATHS"]["INPUT"][table]
    )
    data: pd.DataFrame = pd.read_csv(path)

    # Keep only selected columns
    data = data[cols]

    # Ugly but functional call to ensure correct type conversion
    data = data.convert_dtypes().astype(dtype_map, errors='ignore').convert_dtypes()  # type: ignore

    # Ensure that the product registry dataframe keys are valid
    if not data[keys].notna().all(axis=1).all():
        raise ValueError(f"{table} dataframe keys are missing")
    if not data.value_counts(keys).eq(1).all():
        if duplicate_key_err == 'raise':
            raise KeyError(f"{table} keys are duplicated")
        else:
            print(f"WARN: dupliated keys will be removed from {table}")
            data = data.drop_duplicates(subset=keys, keep=False)

    # Sort product product registry by DIVISION,PRODUCT and reset index
    data = data.sort_values(keys).reset_index(drop=True)

    data = data.rename(columns=name_conversion)

    if "FILTER" in params:
        if table in params["FILTER"]:
            for column, values in params["FILTER"].get(table).to_dict().items():
                column_name = params["FEATURES"].get(column)
                data = data.loc[data[column_name].isin(values)]

    return data

In [4]:
params = LazySettings(settings_files=[config_file])
params = params[config_mode]

In [5]:
players = data_ingestion_basics(params, "PLAYERS")
clubs = data_ingestion_basics(params, "CLUBS")
competitions = data_ingestion_basics(params, "COMPETITIONS")
games = data_ingestion_basics(params, "GAMES")
#international_competitions_stats = data_ingestion_basics(params, "INTERNATIONAL_COMPETITIONS_STATS")
player_valuations = data_ingestion_basics(params, "PLAYER_VALUATIONS")
players = data_ingestion_basics(params, "PLAYERS")
appearances = data_ingestion_basics(params, "APPEARANCES")

## De-normalize data


### Find a player club for a given year
A player is part of a club for a given season if he played at least one match for that club in that season.

In [6]:
# Get appearencs of players in games
app_game = appearances.merge(games, on=[params["FEATURES"]["GAME"]])
#teams = app_game.loc[app_game[params["FEATURES"]["SEASON"]] == season]
teams = app_game[
    [
        params["FEATURES"]["PLAYER"],
        params["FEATURES"]["SEASON"],
        params["FEATURES"]["CLUB"],
        params["FEATURES"]["GAME"],
    ]
]
teams = teams.groupby(
    [
        params["FEATURES"]["PLAYER"],
        params["FEATURES"]["SEASON"],
        params["FEATURES"]["CLUB"],
    ],
    as_index=False,
).count()
teams = teams.rename(columns={params["FEATURES"]["GAME"]: params["FEATURES"]["APPEARANCES"]})

# For each player, keep only the team with most played games
teams = teams.loc[
    teams.groupby(
        [
            params["FEATURES"]["PLAYER"],
            params["FEATURES"]["SEASON"],
        ]
    )[params["FEATURES"]["APPEARANCES"]].idxmax()
]



In [7]:
 players_team = players.merge(teams, on=[params["FEATURES"]["PLAYER"]])

In [8]:
players_team.loc[players_team.player_id == '341092']

,player_id,name,pretty_name,country_of_citizenship,date_of_birth,position,sub_position,foot,height_in_cm,season,club_id,appearances
29730,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2016,430,34
29731,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2017,430,38
29732,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2018,430,41
29733,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2019,430,36
29734,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2020,506,43
29735,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2021,506,18


In [9]:
players_team["year"] = players_team[params["FEATURES"]["SEASON"]]
players_team["month"] = 9
players_team["day"] = 1
players_team["_season_starts"] = pd.to_datetime(players_team[["year", "month", "day"]])
players_team = players_team.drop(columns=["year", "month", "day"])

In [10]:
players_vals = players_team.merge(player_valuations, on=[params["FEATURES"]["PLAYER"]])
players_vals.loc[players_vals.player_id == '341092']
players_vals["_date_diff"] = (players_vals[params["FEATURES"]["DATE"]] - players_vals["_season_starts"]).dt.days
players_vals = players_vals.loc[players_vals["_date_diff"] < 0]



In [11]:
pv_key = [params["FEATURES"]["PLAYER"], params["FEATURES"]["SEASON"]]
players_vals = players_vals[players_vals.groupby(pv_key)["_date_diff"].transform(max) == players_vals["_date_diff"]]
players_lost = len(players_team) - len(players_vals)
players_vals = players_vals.drop(columns=["_date_diff", "_season_starts", params["FEATURES"]["DATE"]])
print(f"{players_lost}/{len(players_team)} players/seasons has been lost when trying to calculate the value.")

3696/55649 players/seasons has been lost when trying to calculate the value.


In [12]:
players_vals = players_vals.sort_values(by=pv_key)
players_vals[params["FEATURES"]["VALUE_DELTA"]] = players_vals[params["FEATURES"]["MARKET_VALUE"]].diff()

In [14]:
players_vals.loc[players_vals.player_id == '341092']

,player_id,name,pretty_name,country_of_citizenship,date_of_birth,position,sub_position,foot,height_in_cm,season,club_id,appearances,market_value,VALUE_DELTA
601565,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2016,430,34,90000,22000
601591,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2017,430,38,9000000,8910000
601616,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2018,430,41,40500000,31500000
601641,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2019,430,36,54000000,13500000
601666,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2020,506,43,43200000,-10800000
601692,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2021,506,18,63000000,19800000
